In [1]:
import gym

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import random

import math

In [2]:
class CNet(nn.Module):
    def __init__(self, in_, out_, hidden=128):
        super(CNet, self).__init__()
        self.mu = nn.Sequential(
            nn.Linear(in_,hidden),
            nn.ReLU(),
            nn.Linear(hidden,out_),
            nn.Tanh()
        )
        self.var = nn.Sequential(
            nn.Linear(in_,hidden),
            nn.ReLU(),
            nn.Linear(hidden,out_),
            nn.Softplus()
        )
        self.value = nn.Sequential(
            nn.Linear(in_,hidden),
            nn.ReLU(),
            nn.Linear(hidden,1)
        )
    
    def forward(self, x):
        return self.mu(x), self.var(x), self.value(x)

In [6]:
env = gym.make("Pendulum-v0")

GAMMA = 0.999
LR = 0.001
EPOCH = 5000

ENT_SCALE = 0.01

BATCH = 4
VIDEO = 400

in_ = env.observation_space.shape[0]
out_ = env.action_space.shape[0]
high_ = env.action_space.high[0]
low_ = env.action_space.low[0]

In [4]:
net = CNet(in_,out_)
opt = optim.Adam(net.parameters(), lr=LR)

In [7]:
batch_cnt = 0
batch_obs = []
batch_act = []
batch_qval = []
for epoch in range(EPOCH):
    obs = env.reset()
    if epoch%VIDEO == 0:
        env.render()
    cur_rew = []
    count = 0
    while True:
        with torch.no_grad():
            mu, var, _ = net(torch.FloatTensor(obs))
            mu = mu.data.numpy()
            sig = torch.sqrt(var).data.numpy()
            act = np.random.normal(mu, sig)
            act = np.clip(act, low_, high_)
        
        next_obs, rew, done, _ = env.step(act)
        if epoch%VIDEO == 0:
            env.render()
        count+=1
        
        batch_obs.append(obs)
        batch_act.append(act)
        cur_rew.append(rew)
        
        obs = next_obs

        if done:
            qval = []
            r_sum = 0
            for r in reversed(cur_rew):
                r_sum*= GAMMA
                r_sum+= r
                qval.append(r_sum)
            batch_qval.extend(list(reversed(qval)))
            batch_cnt+=1
            break
    print("epoch : %d count : %d rew : %d"%(epoch, count, rew))
    
    if batch_cnt == BATCH:
        obss = torch.FloatTensor(batch_obs)
        acts = torch.FloatTensor(batch_act)
        qvals = torch.FloatTensor(batch_qval).unsqueeze(1)

        opt.zero_grad()

        # use network to get logit, value
        mu, var, value = net(obss)

        # value -> need to be similar to qvals.
        value_loss = F.mse_loss(value, qvals)

        # policy gradient. using advatange score to reduce variance.
        first = -((mu - acts)**2/2*var.clamp(low_, high_))
        two = - torch.log(torch.sqrt(2*math.pi*var))
        
        log_prob = first + two
        adv = qvals - value.detach()
        policy_value = adv * log_prob
        policy_loss = -policy_value.mean()

        # using entropy to make agent explore more.
        entropy_loss = ENT_SCALE * (-(torch.log(2*math.pi*var)+1)/2).mean()

        loss = value_loss + policy_loss + entropy_loss
        print(value_loss, policy_loss, entropy_loss)
        loss.backward()

        opt.step()

        batch_cnt = 0
        batch_obs = []
        batch_act = []
        batch_qval = []
env.close()

epoch : 0 count : 200 rew : -2
epoch : 1 count : 200 rew : -4
epoch : 2 count : 200 rew : -5
epoch : 3 count : 200 rew : -14
tensor(112081.9609, grad_fn=<MseLossBackward>) tensor(-185.1898, grad_fn=<NegBackward>) tensor(-0.0373, grad_fn=<MulBackward0>)
epoch : 4 count : 200 rew : -7
epoch : 5 count : 200 rew : -9
epoch : 6 count : 200 rew : -8
epoch : 7 count : 200 rew : -3
tensor(127242.3359, grad_fn=<MseLossBackward>) tensor(-335.8796, grad_fn=<NegBackward>) tensor(-0.0373, grad_fn=<MulBackward0>)
epoch : 8 count : 200 rew : -2
epoch : 9 count : 200 rew : -8
epoch : 10 count : 200 rew : -9
epoch : 11 count : 200 rew : -3
tensor(151251.4844, grad_fn=<MseLossBackward>) tensor(-387.7077, grad_fn=<NegBackward>) tensor(-0.0368, grad_fn=<MulBackward0>)
epoch : 12 count : 200 rew : -1
epoch : 13 count : 200 rew : -14
epoch : 14 count : 200 rew : -7
epoch : 15 count : 200 rew : -3
tensor(104862., grad_fn=<MseLossBackward>) tensor(-19.8714, grad_fn=<NegBackward>) tensor(-0.0372, grad_fn=<MulB

tensor(124575.7734, grad_fn=<MseLossBackward>) tensor(-246.7791, grad_fn=<NegBackward>) tensor(-0.0371, grad_fn=<MulBackward0>)
epoch : 128 count : 200 rew : -4
epoch : 129 count : 200 rew : -3
epoch : 130 count : 200 rew : -8
epoch : 131 count : 200 rew : 0
tensor(117179.6562, grad_fn=<MseLossBackward>) tensor(-186.0582, grad_fn=<NegBackward>) tensor(-0.0370, grad_fn=<MulBackward0>)
epoch : 132 count : 200 rew : -9
epoch : 133 count : 200 rew : -7
epoch : 134 count : 200 rew : -2
epoch : 135 count : 200 rew : -3
tensor(148874.3281, grad_fn=<MseLossBackward>) tensor(-429.5748, grad_fn=<NegBackward>) tensor(-0.0371, grad_fn=<MulBackward0>)
epoch : 136 count : 200 rew : -6
epoch : 137 count : 200 rew : -10
epoch : 138 count : 200 rew : -13
epoch : 139 count : 200 rew : -15
tensor(132410.2188, grad_fn=<MseLossBackward>) tensor(-239.8391, grad_fn=<NegBackward>) tensor(-0.0368, grad_fn=<MulBackward0>)
epoch : 140 count : 200 rew : -5
epoch : 141 count : 200 rew : -9
epoch : 142 count : 200 

epoch : 252 count : 200 rew : -4
epoch : 253 count : 200 rew : -5
epoch : 254 count : 200 rew : 0
epoch : 255 count : 200 rew : -4
tensor(126430.5078, grad_fn=<MseLossBackward>) tensor(-149.6422, grad_fn=<NegBackward>) tensor(-0.0367, grad_fn=<MulBackward0>)
epoch : 256 count : 200 rew : -6
epoch : 257 count : 200 rew : -1
epoch : 258 count : 200 rew : -2
epoch : 259 count : 200 rew : -13
tensor(116541.7109, grad_fn=<MseLossBackward>) tensor(-142.7553, grad_fn=<NegBackward>) tensor(-0.0374, grad_fn=<MulBackward0>)
epoch : 260 count : 200 rew : -10
epoch : 261 count : 200 rew : -10
epoch : 262 count : 200 rew : -15
epoch : 263 count : 200 rew : -5
tensor(142935.0781, grad_fn=<MseLossBackward>) tensor(-455.8052, grad_fn=<NegBackward>) tensor(-0.0366, grad_fn=<MulBackward0>)
epoch : 264 count : 200 rew : -10
epoch : 265 count : 200 rew : -4
epoch : 266 count : 200 rew : 0
epoch : 267 count : 200 rew : -9
tensor(138098.1875, grad_fn=<MseLossBackward>) tensor(-384.7168, grad_fn=<NegBackward

tensor(110048.2500, grad_fn=<MseLossBackward>) tensor(-209.6655, grad_fn=<NegBackward>) tensor(-0.0374, grad_fn=<MulBackward0>)
epoch : 380 count : 200 rew : -11
epoch : 381 count : 200 rew : -2
epoch : 382 count : 200 rew : -2
epoch : 383 count : 200 rew : -2
tensor(116530.3750, grad_fn=<MseLossBackward>) tensor(-171.4978, grad_fn=<NegBackward>) tensor(-0.0372, grad_fn=<MulBackward0>)
epoch : 384 count : 200 rew : -5
epoch : 385 count : 200 rew : -1
epoch : 386 count : 200 rew : -8
epoch : 387 count : 200 rew : -2
tensor(127842.9453, grad_fn=<MseLossBackward>) tensor(-358.5801, grad_fn=<NegBackward>) tensor(-0.0371, grad_fn=<MulBackward0>)
epoch : 388 count : 200 rew : -9
epoch : 389 count : 200 rew : 0
epoch : 390 count : 200 rew : -8
epoch : 391 count : 200 rew : -4
tensor(138256.6094, grad_fn=<MseLossBackward>) tensor(-317.2203, grad_fn=<NegBackward>) tensor(-0.0369, grad_fn=<MulBackward0>)
epoch : 392 count : 200 rew : -9
epoch : 393 count : 200 rew : -7
epoch : 394 count : 200 re

epoch : 504 count : 200 rew : -8
epoch : 505 count : 200 rew : -6
epoch : 506 count : 200 rew : -13
epoch : 507 count : 200 rew : -9
tensor(152376.4062, grad_fn=<MseLossBackward>) tensor(-277.7214, grad_fn=<NegBackward>) tensor(-0.0363, grad_fn=<MulBackward0>)
epoch : 508 count : 200 rew : -6
epoch : 509 count : 200 rew : -10
epoch : 510 count : 200 rew : -9
epoch : 511 count : 200 rew : 0
tensor(123069.1094, grad_fn=<MseLossBackward>) tensor(-171.9602, grad_fn=<NegBackward>) tensor(-0.0372, grad_fn=<MulBackward0>)
epoch : 512 count : 200 rew : -6
epoch : 513 count : 200 rew : -4
epoch : 514 count : 200 rew : -10
epoch : 515 count : 200 rew : -5
tensor(123899.5312, grad_fn=<MseLossBackward>) tensor(-198.2323, grad_fn=<NegBackward>) tensor(-0.0371, grad_fn=<MulBackward0>)
epoch : 516 count : 200 rew : -1
epoch : 517 count : 200 rew : -4
epoch : 518 count : 200 rew : -12
epoch : 519 count : 200 rew : -4
tensor(110076.2656, grad_fn=<MseLossBackward>) tensor(45.7847, grad_fn=<NegBackward>)

...

tensor(126552.4844, grad_fn=<MseLossBackward>) tensor(-304.6810, grad_fn=<NegBackward>) tensor(-0.0374, grad_fn=<MulBackward0>)
epoch : 4484 count : 200 rew : -8
epoch : 4485 count : 200 rew : -1
epoch : 4486 count : 200 rew : -1
epoch : 4487 count : 200 rew : -12
tensor(118896.9453, grad_fn=<MseLossBackward>) tensor(-145.1658, grad_fn=<NegBackward>) tensor(-0.0373, grad_fn=<MulBackward0>)
epoch : 4488 count : 200 rew : -2
epoch : 4489 count : 200 rew : -10
epoch : 4490 count : 200 rew : -2
epoch : 4491 count : 200 rew : -8
tensor(110518.1797, grad_fn=<MseLossBackward>) tensor(81.8681, grad_fn=<NegBackward>) tensor(-0.0375, grad_fn=<MulBackward0>)
epoch : 4492 count : 200 rew : -9
epoch : 4493 count : 200 rew : -6
epoch : 4494 count : 200 rew : -1
epoch : 4495 count : 200 rew : -4
tensor(115547.0781, grad_fn=<MseLossBackward>) tensor(6.6827, grad_fn=<NegBackward>) tensor(-0.0374, grad_fn=<MulBackward0>)
epoch : 4496 count : 200 rew : -1
epoch : 4497 count : 200 rew : -7
epoch : 4498 co

tensor(105779.0938, grad_fn=<MseLossBackward>) tensor(-30.9969, grad_fn=<NegBackward>) tensor(-0.0375, grad_fn=<MulBackward0>)
epoch : 4608 count : 200 rew : -8
epoch : 4609 count : 200 rew : -3
epoch : 4610 count : 200 rew : -5
epoch : 4611 count : 200 rew : -9
tensor(125062.2734, grad_fn=<MseLossBackward>) tensor(-195.7357, grad_fn=<NegBackward>) tensor(-0.0372, grad_fn=<MulBackward0>)
epoch : 4612 count : 200 rew : -2
epoch : 4613 count : 200 rew : -7
epoch : 4614 count : 200 rew : -12
epoch : 4615 count : 200 rew : -12
tensor(98168.8906, grad_fn=<MseLossBackward>) tensor(-61.9440, grad_fn=<NegBackward>) tensor(-0.0377, grad_fn=<MulBackward0>)
epoch : 4616 count : 200 rew : -1
epoch : 4617 count : 200 rew : -8
epoch : 4618 count : 200 rew : -7
epoch : 4619 count : 200 rew : -3
tensor(125647.5312, grad_fn=<MseLossBackward>) tensor(-64.1437, grad_fn=<NegBackward>) tensor(-0.0373, grad_fn=<MulBackward0>)
epoch : 4620 count : 200 rew : -8
epoch : 4621 count : 200 rew : -1
epoch : 4622 c

tensor(120641.7266, grad_fn=<MseLossBackward>) tensor(53.6252, grad_fn=<NegBackward>) tensor(-0.0373, grad_fn=<MulBackward0>)
epoch : 4732 count : 200 rew : -1
epoch : 4733 count : 200 rew : -4
epoch : 4734 count : 200 rew : -2
epoch : 4735 count : 200 rew : -1
tensor(92873.2500, grad_fn=<MseLossBackward>) tensor(179.1321, grad_fn=<NegBackward>) tensor(-0.0376, grad_fn=<MulBackward0>)
epoch : 4736 count : 200 rew : -3
epoch : 4737 count : 200 rew : -2
epoch : 4738 count : 200 rew : -4
epoch : 4739 count : 200 rew : -3
tensor(100276.8359, grad_fn=<MseLossBackward>) tensor(-71.5641, grad_fn=<NegBackward>) tensor(-0.0378, grad_fn=<MulBackward0>)
epoch : 4740 count : 200 rew : -5
epoch : 4741 count : 200 rew : -12
epoch : 4742 count : 200 rew : -13
epoch : 4743 count : 200 rew : -2
tensor(76834.2266, grad_fn=<MseLossBackward>) tensor(435.5743, grad_fn=<NegBackward>) tensor(-0.0376, grad_fn=<MulBackward0>)
epoch : 4744 count : 200 rew : 0
epoch : 4745 count : 200 rew : -1
epoch : 4746 count

tensor(123467.8906, grad_fn=<MseLossBackward>) tensor(-50.3879, grad_fn=<NegBackward>) tensor(-0.0373, grad_fn=<MulBackward0>)
epoch : 4856 count : 200 rew : 0
epoch : 4857 count : 200 rew : -9
epoch : 4858 count : 200 rew : -10
epoch : 4859 count : 200 rew : -5
tensor(151121.3281, grad_fn=<MseLossBackward>) tensor(-397.2973, grad_fn=<NegBackward>) tensor(-0.0371, grad_fn=<MulBackward0>)
epoch : 4860 count : 200 rew : -9
epoch : 4861 count : 200 rew : -10
epoch : 4862 count : 200 rew : -4
epoch : 4863 count : 200 rew : -7
tensor(118563.5625, grad_fn=<MseLossBackward>) tensor(-104.6830, grad_fn=<NegBackward>) tensor(-0.0374, grad_fn=<MulBackward0>)
epoch : 4864 count : 200 rew : 0
epoch : 4865 count : 200 rew : -5
epoch : 4866 count : 200 rew : -5
epoch : 4867 count : 200 rew : -9
tensor(88087.6328, grad_fn=<MseLossBackward>) tensor(276.2357, grad_fn=<NegBackward>) tensor(-0.0376, grad_fn=<MulBackward0>)
epoch : 4868 count : 200 rew : -5
epoch : 4869 count : 200 rew : -7
epoch : 4870 co

tensor(160106.7344, grad_fn=<MseLossBackward>) tensor(-613.3486, grad_fn=<NegBackward>) tensor(-0.0372, grad_fn=<MulBackward0>)
epoch : 4980 count : 200 rew : -10
epoch : 4981 count : 200 rew : -15
epoch : 4982 count : 200 rew : -6
epoch : 4983 count : 200 rew : -14
tensor(109720.6641, grad_fn=<MseLossBackward>) tensor(159.2158, grad_fn=<NegBackward>) tensor(-0.0372, grad_fn=<MulBackward0>)
epoch : 4984 count : 200 rew : -3
epoch : 4985 count : 200 rew : -16
epoch : 4986 count : 200 rew : -8
epoch : 4987 count : 200 rew : -3
tensor(89332.8828, grad_fn=<MseLossBackward>) tensor(188.4363, grad_fn=<NegBackward>) tensor(-0.0374, grad_fn=<MulBackward0>)
epoch : 4988 count : 200 rew : -8
epoch : 4989 count : 200 rew : -6
epoch : 4990 count : 200 rew : -8
epoch : 4991 count : 200 rew : -9
tensor(133379.3281, grad_fn=<MseLossBackward>) tensor(-209.4252, grad_fn=<NegBackward>) tensor(-0.0372, grad_fn=<MulBackward0>)
epoch : 4992 count : 200 rew : -5
epoch : 4993 count : 200 rew : -9
epoch : 499